In [5]:
# Always reload modules to have the current version
%reload_ext autoreload
%autoreload 2

In [6]:
from ranking.models.bm25 import BM25Model
from ranking.storage.document_store import DocumentStore
from ranking.util import dataset_paths as dp
from ranking.util import json_lines as jl
import dask.dataframe as dd
import ranking.evaluation.evaluate as ev

store = DocumentStore(dp.lemmatized_unique_functions_corpus)
eval_set = jl.read_jsonl(dp.unified_hoogle_results)
ddf = dd.from_pandas(eval_set, npartitions=16)
model = BM25Model(store)

In [7]:
bm25_result = ddf.apply(
    lambda row: model.rank(row['docQuery'].split(), row['hoogleRes']),
    axis=1,
    meta=('result', int)
).compute()
eval_set['bm25res'] = bm25_result 
eval_set['bm25Rank'] = eval_set.apply(lambda row: ev.get_rank(row['storageId'], row['bm25res']), axis=1)

In [8]:
print('Hoogle + BM25:')
print(ev.evaluate(eval_set['bm25Rank']))

Hoogle + BM25:
   Total  TopOne Total:  TopOne in %  TopTen Total:  TopTen in %       MRR
0   1000            871        0.871            979        0.979  0.920583
